In [19]:
def google_sector_report():
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.google.com/finance"
    response = requests.get(url)
    page_data = BeautifulSoup(response.content,'lxml')
    
    json_dump={}
    p_change=[]     # change of each sector
    for tag_1 in page_data.find_all('div',id='secperf'):
        for thing in tag_1.find_all('span'):
            p_change.append(thing.text)
    
    s_url=[]
    for tag_1 in page_data.find_all('div',id='secperf'):
        for thing in tag_1.find_all('a'):
            link = thing.get('href')
            link = 'https://www.google.com' + link
            s_url.append(link)
    
    results={}
    s_name=[]       # name of each sector

    biggest_gainer=[]
    biggest_loser=[]
    for i in range(len(s_url)):
        response = requests.get(s_url[i])
        if response.status_code != 200:
            json_dump['STATUS']='BAD'
            import json
            final=json.dumps(json_dump,sort_keys=True)
            return final
        page_data = BeautifulSoup(response.content,'lxml')

        s_data={}
        b_g={}
        b_l={}
        s_data['change']=float(p_change[i][0:-1])     # extract percentage change

        for tag in page_data.find_all('div',{"class":"hdg top appbar-hide"}):
            s_name.append(tag.find('h3').text[1:-1])
        for tag in page_data.find_all('table',{"class":"topmovers"}):
            c_name=[]
            c_change=[]
            for thing in tag.find_all('a'):
                c_name.append(thing.text)
            for thing in tag.find_all('span'):
                c_change.append(thing.text)
            b_g['change']=float(c_change[1][1:-2])
            b_g['equity']=c_name[0]
            b_l['change']=float(c_change[11][1:-2])
            b_l['equity']=c_name[10]
            s_data['biggest_gainer']=b_g
            s_data['biggest_loser']=b_l
            results[s_name[i]]=s_data
    
    json_dump['results']=results
    json_dump['STATUS']='GOOD'
    import json
    final=json.dumps(json_dump,sort_keys=False)
    return final

In [20]:
google_sector_report()

'{"results": {"Energy": {"change": 0.47, "biggest_gainer": {"change": 6.34, "equity": "Contango Oil & Gas Co."}, "biggest_loser": {"change": -10.7, "equity": "CIRCOR Intl., Inc."}}, "Non-Cyclical Consumer Goods & Services": {"change": 0.42, "biggest_gainer": {"change": -3.92, "equity": "Weight Watchers Intl"}, "biggest_loser": {"change": 0.42, "equity": "Procter & Gamble Co"}}, "Cyclical Consumer Goods & Services": {"change": -0.27, "biggest_gainer": {"change": 15.76, "equity": "Zumiez Inc."}, "biggest_loser": {"change": -10.19, "equity": "Cato Corp"}}, "Technology": {"change": 0.18, "biggest_gainer": {"change": 5.92, "equity": "AIXTRON SE (ADR)"}, "biggest_loser": {"change": -10.05, "equity": "Park Electrochemical Corp"}}, "Healthcare": {"change": -0.63, "biggest_gainer": {"change": 14.86, "equity": "ICU Medical, Incorporated"}, "biggest_loser": {"change": -48.49, "equity": "Alnylam Pharmaceuticals"}}, "Telecommunications Services": {"change": -0.53, "biggest_gainer": {"change": 3.07,